In [1]:
%load_ext dotenv
%dotenv

In [22]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql import functions as F
from pyspark.sql import Window

# noinspection PyUnresolvedReferences
import human.plotly_template

pio.templates.default = "plotly+human"

In [3]:
df = pd.read_parquet('s3://merged-tweets/testing-sample/test.parquet')

# Dataset exploration

In [11]:
df = ps.from_pandas(df).to_spark()

/home/ahenao/miniconda3/envs/mineria-env/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [12]:
df.show(10)

+-------------------+-------------------+--------------------+----+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
|         created_at|                 id|           full_text| geo|coordinates|retweet_count|favorite_count|reply_count|quote_count|favorited|retweeted|possibly_sensitive|lang|
+-------------------+-------------------+--------------------+----+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
|2018-05-25 13:19:14|1000003359850598400|‘Ukraine war on f...|null|       null|            0|             0|          0|          0|    false|    false|             false|  en|
|2018-05-25 13:32:56|1000006805269958656|'Ukraine war on f...|null|       null|            0|             0|          0|          0|    false|    false|             false|  en|
|2018-05-25 13:49:24|1000010951305854976|UKRAINE WAR: Kiev...|null|       null|            1|             0|       

In [13]:
df.dtypes

[('created_at', 'timestamp'),
 ('id', 'bigint'),
 ('full_text', 'string'),
 ('geo', 'void'),
 ('coordinates', 'void'),
 ('retweet_count', 'bigint'),
 ('favorite_count', 'bigint'),
 ('reply_count', 'bigint'),
 ('quote_count', 'bigint'),
 ('favorited', 'boolean'),
 ('retweeted', 'boolean'),
 ('possibly_sensitive', 'boolean'),
 ('lang', 'string')]

In [16]:
df.count()

1000

In [19]:
z = df.count()
(df.replace(float('nan'), None)
 .agg(*[F.expr(f'count({col})/{z} as {col}') for col in df.columns])
).show()

+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
|created_at| id|full_text|geo|coordinates|retweet_count|favorite_count|reply_count|quote_count|favorited|retweeted|possibly_sensitive|lang|
+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
|       1.0|1.0|      1.0|0.0|        0.0|          1.0|           1.0|        1.0|        1.0|      1.0|      1.0|             0.875| 1.0|
+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+



In [23]:
w = Window.partitionBy('id')
df.select('*', F.count('id').over(w).alias('dupeCount'))\
    .where('dupeCount > 1')\
    .drop('dupeCount')\
    .show()

+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
|created_at| id|full_text|geo|coordinates|retweet_count|favorite_count|reply_count|quote_count|favorited|retweeted|possibly_sensitive|lang|
+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+
+----------+---+---------+---+-----------+-------------+--------------+-----------+-----------+---------+---------+------------------+----+



In [24]:
df.groupBy('lang').count().show()

+----+-----+
|lang|count|
+----+-----+
|  en|  977|
| zxx|    5|
|  sv|    1|
|  ru|    4|
|  uk|    2|
|  ro|    1|
|  ht|    1|
|  et|    2|
| und|    6|
|  de|    1|
+----+-----+



In [25]:
df.groupBy('favorited').count().show()


+---------+-----+
|favorited|count|
+---------+-----+
|    false| 1000|
+---------+-----+



In [26]:
df.groupBy('retweeted').count().show()

+---------+-----+
|retweeted|count|
+---------+-----+
|    false| 1000|
+---------+-----+



In [27]:
df.groupBy('possibly_sensitive').count().show()


+------------------+-----+
|possibly_sensitive|count|
+------------------+-----+
|              null|  125|
|             false|  865|
|              true|   10|
+------------------+-----+



# Descriptive analysis

## Daily Tweets

In [43]:
df_plot = (df.select(F.date_format('created_at','yyyy-MM-dd').alias('created_at'))
 .groupby('created_at')
 .count()
 .withColumnRenamed('count', 'size'))
px.line(df_plot.toPandas().sort_values(by="created_at"), x='created_at', y='size')

## Monthly Tweets

In [45]:
df_plot = (df.select(F.date_format('created_at', 'yyyy-MM').alias('created_at'))
 .groupby('created_at')
 .count()
 .withColumnRenamed('count', 'size'))
px.line(df_plot.toPandas().sort_values(by="created_at"), x='created_at', y='size')

## Retweets and favs distribution

### Including 0

In [52]:
count, limits = np.histogram(df.select('retweet_count').toPandas(), bins=2000)
fig = px.histogram(x=(limits[:-1] + limits[1:]) / 2 + 2, y=count, nbins=2000, range_x=[0, 100])
fig.update_xaxes(title='retweet_count').update_yaxes(title='count')

In [53]:
count, limits = np.histogram(df.select('favorite_count').toPandas(), bins=4000)
fig = px.histogram(x=(limits[:-1] + limits[1:]) / 2 + 2, y=count, nbins=4000, range_x=[0, 100])
fig.update_xaxes(title='favorite_count').update_yaxes(title='count')

### Excluding 0

In [62]:
count, limits = np.histogram(df.filter(df.retweet_count > 0).select("retweet_count").toPandas(), bins=2000)
fig = px.histogram(x=(limits[:-1] + limits[1:]) / 2 + 2, y=count, nbins=2000, range_x=[0, 100])
fig.update_xaxes(title='retweet_count').update_yaxes(title='count')

In [63]:
count, limits = np.histogram(df.filter(df.favorite_count > 0).select("favorite_count").toPandas(), bins=4000)
fig = px.histogram(x=(limits[:-1] + limits[1:]) / 2 + 2, y=count, nbins=4000, range_x=[0, 100])
fig.update_xaxes(title='favorite_count').update_yaxes(title='count')

# Text Analysis

In [65]:
import spacy
from collections import Counter

In [66]:
nlp = spacy.load('en_core_web_sm')

In [72]:
text_array = np.array(df.select("full_text").collect()).flatten().tolist()

In [87]:
docs = nlp.pipe(text_array) # This will require a better handling -> https://spacy.io/usage/processing-pipelines

In [75]:
# This preprocessing can be done in the dataframe (enabling the comparison between dask and pyspark)
def is_token_allowed(token, pos_tag=None):
    matches_pos = True if pos_tag is None else token.pos_ == pos_tag
    if (not token or not token.text.strip() or
            token.is_stop or token.is_punct or not matches_pos):
        return False
    return True


def preprocess_token(token, lemma=False):
    if lemma:
        return token.lemma_.strip().lower()
    return token.text.strip().lower()

## No lemmatization

In [88]:
words = [preprocess_token(token) for token in doc if is_token_allowed(token) for doc in docs]
nouns = [preprocess_token(token) for token in doc if is_token_allowed(token, pos_tag='NOUN') for doc in docs]
verbs = [preprocess_token(token) for token in doc if is_token_allowed(token, pos_tag='VERB') for doc in docs]

In [78]:
word_freq = Counter(words)
common_words = word_freq.most_common(50)

noun_freq = Counter(nouns)
common_nouns = noun_freq.most_common(50)

verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(50)

### Word Count

In [79]:
df_plot = pd.DataFrame(common_words, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)

### Noun count

In [80]:
df_plot = pd.DataFrame(common_nouns, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)

### Verb count

In [81]:
df_plot = pd.DataFrame(common_verbs, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)

## Lemmatization

In [82]:
words = [preprocess_token(token, lemma=True) for token in doc if is_token_allowed(token) for doc in docs]
nouns = [preprocess_token(token, lemma=True) for token in doc if is_token_allowed(token, pos_tag='NOUN') for doc in docs]
verbs = [preprocess_token(token, lemma=True) for token in doc if is_token_allowed(token, pos_tag='VERB') for doc in docs]

In [83]:
word_freq = Counter(words)
common_words = word_freq.most_common(50)

noun_freq = Counter(nouns)
common_nouns = noun_freq.most_common(50)

verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(50)

### Word Count

In [84]:
df_plot = pd.DataFrame(common_words, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)

### Noun count

In [85]:
df_plot = pd.DataFrame(common_nouns, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)

### Verb count

In [86]:
df_plot = pd.DataFrame(common_verbs, columns=['word', 'count'])
px.bar(df_plot.sort_values('count'), x='count', y='word', text_auto=True, height=1000)